# Import Package

In [8]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words('indonesian') + list(punctuation) + ['``']

# Import Data

In [10]:
df = pd.read_csv('kompas.csv')
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

# TFIDF Similarity => Document Similarity

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00757052, 0.00998045, ..., 0.00857271, 0.00633928,
        0.01502023]])

In [17]:
sim.argsort()

array([[1131,  932, 1593, ...,  215,  144,    0]], dtype=int64)

In [18]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [19]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [22]:
df.teks[215][:215]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pagi, mereka mengi'

In [21]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

# Keyword Extraction

In [25]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [26]:
tfidf_matrix[0].toarray()

array([[0.02117327, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [28]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274178, 365425, 365424, ..., 384924, 435631, 165984], dtype=int64)

In [29]:
vocab[165984]

'ginandjar'

In [30]:
vocab[435631]

'putusan'

In [31]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

# Make a Function

In [32]:
def keyword_extract(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sort_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sort_tfidf[-10:])]

# Sample for Article

In [34]:
text = """
 Saat ini Indonesia hanya memiliki cadangan bahan bakar minyak (BBM) operasional untuk 18-22 hari. Artinya jika ada sesuatu bencana besar atau perang, dan pasokan minyak dari luar negeri terhenti, stok BBM Indonesia hanya bisa bertahan paling lama 22 hari. Dibandingkan dengan negara-negara lain, cadangan BBM Indonesia tergolong amat sedikit. Malaysia punya cadangan BBM untuk 25 hari, Singapura punya cadangan BBM untuk 90 hari, cadangan BBM China juga 90 hari. Bahkan Amerika Serikat (AS) punya cadangan BBM hingga 260 hari. Jepang juga punya cadangan BBM yang sangat kuat, setara dengan kebutuhannya selama 7 bulan. 'Saya pernah ke Jepang, mereka punya cadangan BBM nasional setara dengan kebutuhan 7 bulan. Makanya ketika dia (Jepang) tsunami, enggak ada masalah,' kata Kepala BPH Migas, Fanshurullah Asa, kepada kumparan (kumparan.com), Kamis (5/10).  Pria yang akrab dipanggil Ifan ini menuturkan, tangki untuk penimbunan cadangan BBM di Jepang sangat besar, pesawat Boeing 737 pun bisa tenggelam di dalamnya'Saya pernah masuk ke sana, besar sekali. Pesawat Boeing 737 kalau masuk ke tangkinya juga tenggelam di dalamnya,' ujarnya. Cadangan BBM nasional Indonesia saat ini masih jauh dari jumlah ideal. Di Eropa, rata-rata tiap negara menyimpan cadangan BBM untuk 4 bulan. 'Kalau standar Eropa 4 bulan,' tukas Ifan. Untuk meningkatkan cadangan BBM nasional, saat ini Kementerian ESDM sedang menyiapkan Peraturan Menteri ESDM. Isinya mewajibkan setiap badan usaha yang berbisnis penyaluran BBM seperti Pertamina, Shell, Total, dan AKR untuk menyimpan cadangan operasional untuk 25 hari di Indonesia.'Kalau kita bisa sampai 25 hari saja sudah bagus. Sekarang saya sudah paraf drafnya (Permen ESDM). Saya, Pak Sekjen, Pak Dirjen Migas sudah paraf, sekarang masuk ke Pak Menteri (Ignasius Jonan). Rencananya akan dikeluarkan Permen-nya, isinya setiap badan usaha wajib mempunyai cadangan operasional 25 hari,' paparnya. Aturan ini berlaku paling lambat 1 Januari 2019. Pertamina hingga AKR diberi waktu kurang lebih 1 tahun untuk menyiapkan tangki-tangki timbun yang bisa menyimpan cadangan opersional sebesar kebutuhan 25 hari.Cadangan BBM tersebut dapat digunakan ketika sewaktu-waktu terjadi keadaan darurat di dalam negeri. Ini penting buat ketahanan energi nasional. 'Berlaku paling lambat 1 Januari 2019. Cadangan operasional ini akan jadi cadangan BBM nasional. Jadi ketika terjadi bencana, bisa dipakai untuk kepentingan nasional,' tutupnya.
"""

In [35]:
keyword_extract(text, tfidf)

['cadangan',
 'bbm',
 'bbm nasional',
 'esdm',
 'operasional',
 '25',
 "'kalau",
 'paraf',
 'ifan',
 'jepang']